* 쿠팡와우,마켓컬리,SSG 새벽심야배송하는 3개업체 비교,각 업체마다 특징이 나타나는지,제품의 특징이 나타나는지

In [1]:
pip install selenium 

Note: you may need to restart the kernel to use updated packages.


In [23]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

from selenium import webdriver

driver = webdriver.Chrome("chromedriver.exe")

import time

In [24]:
#인스타 접속
driver.get('https://www.instargram.com')
time.sleep(5) #시간을 넉넉히 설정합니다.

In [25]:
#로그인에 필요한 사항을 브라우저 제어를통해 입력합니다.
email = '01086897540'   ### 사용할 계정 정보 입력
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'rkdgus12!@' ### 비번 정보 수정
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)


In [28]:
word = "마켓컬리" #검색할 해시태그. 띄어쓰기를 사용하면 안됩니다
url = insta_searching(word)
driver.get(url)
time.sleep(2)

In [29]:
#첫게시물을 여는 함수를 정의합니다
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

In [30]:
import re
from bs4 import BeautifulSoup

#내용을 가져오는 함수를 정의합니다.
def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    

    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 정규표현식을 활용하여 해시태그를 가져옵니다.
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자를 가져옵니다.
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 수를 가져옵니다.
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치정보를 가져옵니다.
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # 수집한 정보를 리스트로 저장합니다.
    data = [content, date, like, place, tags]
    return data



In [31]:
#다음게시글을 여는 함수를 정의합니다.
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

In [32]:
#함수를 차례대로 실행합니다.
select_first(driver)
results = [ ]
target = 50      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시 2초 대기후 다음글로 넘어갑니다.
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)


In [33]:
#result적당히 출력해보기
print(results[:2])




[['2021.03.15.월-#아보카도김치볶음밥#치즈돈까스#해쉬브라운-세상 맛있어서 이 조합으로 한 번 더 해먹기 향 찐한 아보카도까지 올려주니 💛🥑💛치즈돈까스는 내사랑 #마켓컬리 에서샀는데모짜렐라&체다 치즈가 듬뿍 들어있어요즘 빼놓지 않고 주문하는 중 (매우 추천)', '2021-03-15', 0, '', ['#아보카도김치볶음밥', '#치즈돈까스', '#해쉬브라운-세상', '#마켓컬리']], ['#여니식단_점심⠀- 흑미밥 1/2공기- 계란2알 + 체다치즈 1/2 + 크래미 1개- #존쿡델리미트 비엔나- 김치찜 3점- 멀티그레인 무화과빵 1조각- #모모스커피 3모금⠀점심은 집에 있는 재료로호다닥 차려먹었어요 :)⠀근데 존쿡델리미트 제품들은 전부 짜다는 걸제가 깜박했지 뭐에요...⠀뜨거운 물에 잠시 담가놨는데도저한테는 짜던 비엔나씨...⠀전체적으로 밥량에 비해 짜서입가심으로 빵이랑 커피쓰를 먹어줬어요!⠀저희 집은 근처에 있는 모모스 커피에서항상 드립커피를 사다먹는데이제 다른 커피는 못 먹는 입이 되버린..ㅎㅎ⠀커피 좋아하시는 분들은#마켓컬리 에서도 팔고 있으니한 번 드셔보세용♥️⠀그럼 오늘도 행복한 하루 되시길 바라며우리 저녁에 다시 만나용 🙌🏻⠀', '2021-02-28', 0, 'Busan 釜山, South Korea', ['#여니식단_점심⠀-', '#존쿡델리미트', '#모모스커피', '#마켓컬리']]]


In [34]:
#데이터프레임 만들고 엑셀로 저장하기
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('crawling_sample_1.xlsx', index=False) 